In [1]:
%%capture
from sys import path
path.insert(0, '../../static')
from agyw import AgywPrev
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
from pandas import (
    DataFrame,
    read_sql_query,
    ExcelWriter
)
import openpyxl
from enum import Enum
from datetime import  datetime


# Parentalite
---

1. pilotage de lotin/kenskoff

In [2]:
%%capture
datim = AgywPrev()
base = datim.data_dreams_valid

In [3]:
base.id_patient.count()

9943

In [4]:
base.pivot_table(index='age_range', values='id_patient',columns='departement',aggfunc='count',fill_value=0, margins=True)

departement,Artibonite,Nord,Ouest,All
age_range,,,,
10-14,1678,413,1774,3865
15-19,1651,569,1882,4102
20-24,787,302,887,1976
All,4116,1284,4543,9943


In [5]:
actif_parenting = base[
    (base.parenting_detailed!="no")
]

actif_parenting.pivot_table(
    index="commune",
    columns="parenting_detailed",
    values='id_patient',
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name="Grand Total"                             
)


all_kenscoff = actif_parenting[
    (actif_parenting.commune == "Kenscoff")
]

kenscoff_completed = actif_parenting[
    (actif_parenting.commune == "Kenscoff")&
    (actif_parenting.parenting_detailed == "yes")
]

kenscoff_incompleted = actif_parenting[
    (actif_parenting.commune == "Kenscoff")&
    (actif_parenting.parenting_detailed == "has_started")
]

all_kenscoff.id_patient.count()
kenscoff_completed.id_patient.count()
kenscoff_incompleted.id_patient.count()

parenting_detailed,has_started,yes,Grand Total
commune,,,
Cap-Haïtien,18,0,18
Delmas,0,1,1
Desdunes,1,30,31
Dessalines,25,2,27
Grande Saline,4,2,6
Gressier,8,3,11
Kenscoff,12,34,46
La Chapelle,0,1,1
Liancourt,9,0,9


46

34

12

## SUR HIV

In [6]:
load_dotenv()
# get the environment variables needed
USER = config('USRCaris')
PASSWORD = config('PASSCaris')
HOSTNAME = config('HOSTCaris')
DBNAME = config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

class Period(Enum):
    date = '2022-01-01'
    wb=''
    date_du_jour = datetime.today().strftime("%d_%m_%Y")


query = f"""
SELECT
	a.*,
	GROUP_CONCAT(distinct topic, ',')as concat_topic,
	COUNT(*) AS qte_by_creator
FROM (
	SELECT
		dpga.id_patient AS id_patient,
		dpgs.id_group AS id_session,
		dpgs.facilitator,
		dpgs.facilitator1,
		date,
		topic,
		dpgs.created_by,
		lc.name AS commune,
		ld.name AS departement,
		dh.name AS hub,
		au.email AS email,
		au.username,
		b.first_name,
		b.last_name,
		p.patient_code AS dreams_code
	FROM
		caris_db.dream_parenting_group_session dpgs
	LEFT JOIN dream_parenting_group_attendance dpga ON dpgs.id = dpga.id_parenting_group_session
	LEFT JOIN dream_group dg ON dpgs.id_group = dg.id
	LEFT JOIN dream_hub dh ON dg.id_dream_hub = dh.id
	LEFT JOIN lookup_commune lc ON lc.id = dh.commune
	LEFT JOIN lookup_departement ld ON ld.id = lc.departement
	LEFT JOIN auth_users au ON au.id = dpgs.created_by
	LEFT JOIN beneficiary b ON dpga.id_patient = b.id_patient
	LEFT JOIN patient p ON p.id = dpga.id_patient
WHERE (((dpga.parent_g = 'P')
		OR(dpga.parent_vd = 'P')
		OR(dpga.yg_g = 'P')
		OR(dpga.yg_vd = 'P'))
	AND(dpgs.date >= '{Period.date.value}'))) a
GROUP BY
	a.id_patient, a.username

"""


parenting_control= read_sql_query(query, engine, parse_dates=True)



# close the pool of connection
engine.dispose()

True

In [7]:
parenting_control.id_patient.count()

33

In [8]:
all_kenscoff = all_kenscoff[['id_patient', 'nbre_parenting_coupe_present','id_parenting_group',  'age_in_year' ,'age_range', 'ovc_age', 'date_interview', 'month_in_program_range']]
all_kenscoff.id_patient.count()

46

In [9]:
la_cohorte_de_lotin = parenting_control.merge(all_kenscoff,on='id_patient',how="left")
la_cohorte_de_lotin

,id_patient,id_session,facilitator,facilitator1,date,topic,created_by,commune,departement,hub,...,dreams_code,concat_topic,qte_by_creator,nbre_parenting_coupe_present,id_parenting_group,age_in_year,age_range,ovc_age,date_interview,month_in_program_range
0,76370,3783,Derise Marie Weslaine,Derise Marie Weslaine,2022-02-14,6,1553,Kenscoff,Ouest,Lotin,...,PAP/DRMS/032035720,"1,,2,,3,,4,,5,,6,,7,,10,,11,,12,,13,,14,",12,12,3783.0,17,15-19,15-17,2020-01-06,25+ months
1,76624,3783,Derise Marie Weslaine,Derise Marie Weslaine,2022-02-14,7,1553,Kenscoff,Ouest,Lotin,...,PAP/DRMS/032089910,"1,,2,,3,,4,,5,,6,,7,,8,,9,,10,,11,,12,,13,,14,",14,14,3783.0,16,15-19,15-17,2020-02-10,25+ months
2,76637,3820,Derise Marie Weslaine,Derise Marie Weslaine,2022-03-07,14,1553,Kenscoff,Ouest,Lotin,...,PAP/DRMS/032089913,"1,,2,,8,,9,,12,,13,,14,",7,7,3820.0,14,10-14,10-14,2020-01-10,25+ months
3,76656,3783,Derise Marie Weslaine,Derise Marie Weslaine,2022-02-14,6,1553,Kenscoff,Ouest,Lotin,...,PAP/DRMS/032089920,"1,,2,,3,,4,,5,,6,,7,,8,,9,,10,,11,,12,,13,,14,",14,14,3783.0,15,15-19,15-17,2019-11-28,25+ months
4,78772,3820,Derise Marie Weslaine,Derise Marie Weslaine,2022-02-03,1,1553,Kenscoff,Ouest,Lotin,...,PAP/DRMS/032117017,"1,,2,,6,,8,,9,,10,,11,,12,,13,,14,",10,10,3820.0,13,10-14,10-14,2019-11-29,25+ months
5,78885,3820,Derise Marie Weslaine,Derise Marie Weslaine,2022-02-21,9,1553,Kenscoff,Ouest,Lotin,...,PAP/DRMS/032117028,"1,,2,,6,,8,,9,,10,,11,,12,,13,,14,",10,10,3820.0,14,10-14,10-14,2020-01-06,25+ months
6,81999,3854,Derise Marie Weslaine,Derise Marie Weslaine,2022-02-21,8,1553,Kenscoff,Ouest,Lotin,...,PAP/DRMS/032035732,"1,,2,,3,,4,,5,,6,,7,,8,,9,,10,,11,,12,,13,,14,",14,14,3854.0,13,10-14,10-14,2019-11-19,25+ months
7,82174,3783,Derise Marie Weslaine,Derise Marie Weslaine,2022-03-07,14,1553,Kenscoff,Ouest,Lotin,...,PAP/DRMS/032137113,"1,,2,,3,,4,,5,,6,,7,,8,,9,,10,,11,,12,,13,,14,",14,14,3783.0,16,15-19,15-17,2020-02-03,25+ months
8,82856,3820,Derise Marie Weslaine,Derise Marie Weslaine,2022-02-03,1,1553,Kenscoff,Ouest,Lotin,...,PAP/DRMS/032146535,"1,,2,,6,,8,,9,,10,,11,,12,,13,,14,",10,10,3820.0,15,15-19,15-17,2018-09-24,25+ months
9,84459,3820,Derise Marie Weslaine,Derise Marie Weslaine,2022-02-03,2,1553,Kenscoff,Ouest,Lotin,...,PAP/DRMS/130163010,"1,,2,,6,,8,,9,,10,,11,,12,,13,,14,",10,10,3820.0,16,15-19,15-17,2018-06-05,25+ months


In [10]:
la_cohorte_de_lotin.to_excel("la_cohorte_de_lotin.xlsx",index=False,na_rep="")